<a href="https://colab.research.google.com/github/eujin99/real-estate-support-train/blob/main/hoon_real_estate_train_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 필요 라이브러리 설치

In [ ]:
pip install -U accelerate==0.29.3 peft==0.10.0 bitsandbytes==0.43.1 transformers==4.40.1 trl==0.8.6  datasets==2.19.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 118.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import os
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer


In [ ]:
import huggingface_hub
huggingface_hub.login()

2. 모델 설정 (본인 hugging dataset 폴더 참고 ★)

In [ ]:
# Hugging Face Basic Model 한국어 모델
base_model = "beomi/Llama-3-Open-Ko-8B"

# Custom Dataset ★ 본인이 hugging face 내 저장한 모델경로를 설정해야함 ★
eujincode_dataset = "chloestella/hoon_qa"

new_model = "Llama-eujin-real-estate-Ko-3-8B"

In [ ]:
dataset = load_dataset(eujincode_dataset, split="train")

# 데이터 확인
print( dataset[28] )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/355 [00:00<?, ? examples/s]

{'question': '역세권 매물을 추천해줘.', 'answer': '인천 미추홀구 주안동 111-1에 위치한 우리들애(op)은 역세권에 있으며, 매매가는 2,990,033,500만원입니다.'}


4. 4비트 양자화 QLoRa 파인튜닝(효율성) * 파라미터 고정 시키고 추가데이터만 튜닝

In [ ]:
# 일단 flash_attn install 과정 너무 오래걸려서 생략
# 메모리 성능(속도)에만 영향 줘서 일단은 보류했슴..
# 결과겂애는 영향 안 미치는 걸로 판단

if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

# QLoRA config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=False,
)


5. 라마2모델 불러오기

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
    # device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

...

model-00004-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/1.29G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

6. 토크나이저 불러오기 (Hugginface에서 토크나이저를 로드하고 padding_side를 "right"로 설정하여 fp16과 관련된 문제해결)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 7. PEFT 파라미터 (Parameter-Efficient Fine-Tuning (PEFT)은 모델 파라미터의 작은 하위 집합만 업데이트)

https://huggingface.co/docs/peft/conceptual_guides/lora

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

8. Training parameters

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
# 데이터셋 로드
eujincode_dataset = "chloestella/hoon_qa"
dataset = load_dataset(eujincode_dataset)

# 'text' 컬럼 생성
def create_text_column(example):
    example['text'] = f"question: {example['question']} answer: {example['answer']}"
    return example

# 'train' split에 'text' 컬럼 추가
dataset['train'] = dataset['train'].map(create_text_column)

Map:   0%|          | 0/355 [00:00<?, ? examples/s]

9. model 파인튜닝

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/355 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss
25,2.457800
50,1.507200
75,1.281200
100,1.233300
125,1.098500
150,1.057800
175,0.993500
200,0.794700
225,0.767400
250,0.687900


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=890, training_loss=0.6334781496712331, metrics={'train_runtime': 603.3664, 'train_samples_per_second': 5.884, 'train_steps_per_second': 1.475, 'total_flos': 8861513317244928.0, 'train_loss': 0.6334781496712331, 'epoch': 10.0})

In [ ]:
prompt = "실입금 3000만원 이하의 매물은?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)
print(result[0]['generated_text'])


실입금 3000만원 이하의 매물은? 3000만원 이하의 실입으로 가능한 매물은 광주시 목현동 154-12에 위치한 다이브씨티(op)은 실입 3000만원으로 가능합니다. 제곱미터당 매입가는 349,500만원입니다. 금융가 가능합니다. 목현초, 광주중학교에 접근할 수 있습니다. 집근처에 파크워드(op)은 실입 3000만원으로 가능합니다. 제곱미터당 매입가은 349,500만원입니다. 금융가 가능합니다. 목현초, 광주중학교에 접근할 수 있습니다. 집근처에 파크워드(op)은 실입 3000만원으로 가능합니다. 제곱미터당 매입가은 349,500만원입니다. 금융가 가능합니다. 목현초, 광주중


In [ ]:
# 모델을 로컬 디렉토리에 저장
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")


('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/tokenizer.json')

In [ ]:
# Hugging Face Hub에 업로드
model_name = "Llama-eujin-real-estate-Ko-3-8B"
model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.24G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/chloestella/Llama-eujin-real-estate-Ko-3-8B/commit/94c4fa41ef60eb5a680a5c921a62cee50eb64420', commit_message='Upload tokenizer', commit_description='', oid='94c4fa41ef60eb5a680a5c921a62cee50eb64420', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
### colab에서 메모리 문제로 저장 error

In [ ]:
# # Flush memory
# del trainer, model
# gc.collect()
# torch.cuda.empty_cache()

# # Reload tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(base_model)
# # model = AutoModelForCausalLM.from_pretrained(
# #     base_model,
# #     low_cpu_mem_usage=True,
# #     return_dict=True,
# #     torch_dtype=torch.float16,
# #     device_map="auto",
# # )

# model = AutoModelForCausalLM.from_pretrained(
#     base_model,
#     quantization_config=quant_config,
#     device_map={"": 0},
#     torch_dtype=torch.float16
#     # device_map="auto"
# )

# model, tokenizer = setup_chat_format(model, tokenizer)

# # Merge adapter with base model
# model = PeftModel.from_pretrained(model, new_model)
# model = model.merge_and_unload()
# model.push_to_hub(new_model, use_temp_dir=False)
# tokenizer.push_to_hub(new_model, use_temp_dir=False)